In [454]:
import pandas as pd
import pandas_profiling
import numpy as np
import math
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import datasets
from datetime import date, datetime
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm, tqdm_notebook
import matplotlib.pyplot as plt
plt.style.use("seaborn")
sns.set(style="white")

## Perfil de peliculas y usuarios.

### Perfil de las peliculas segun su genero 

In [2]:
df_genres = pd.read_csv("./data/movie_genres.csv")
df_movies = pd.read_csv("./data/movies.csv")
df_ratings = pd.read_csv("./data/ratings_train.csv")

In [3]:
#Generos para todas las peliculas
df_genres["genre"].unique()

array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'IMAX', 'Documentary', 'War', 'Musical',
       'Film-Noir', 'Western', 'Short'], dtype=object)

In [4]:
# Localizo los IDs de las peliculas de Action
mids = df_genres.loc[df_genres["genre"]=="Action", "movieID"].unique()
mids

array([    6,     9,    10, ..., 64921, 64997, 64999])

In [5]:
# Agrego una columna donde me identifica cuales son de accion y cuales no
df_movies["Genre Action"] = df_movies["id"].isin(mids).astype("int16")
df_movies.head()
#df_movies["Genre Action"].sum()

,id,title,year,imdbID,rtID,Genre Action
0,1,Toy story,1995,114709,toy_story,0
1,2,Jumanji,1995,113497,1068044-jumanji,0
2,3,Grumpy Old Men,1993,107050,grumpy_old_men,0
3,4,Waiting to Exhale,1995,114885,waiting_to_exhale,0
4,5,Father of the Bride Part II,1995,113041,father_of_the_bride_part_ii,0


In [6]:
# Creo una matriz donde se agrego un flag "1" en cada genero del que la pelicula pertenece.
# Perfil de la pelicula sobre el Genero
for g in df_genres["genre"].unique():
    mids = df_genres.loc[df_genres["genre"] == g, "movieID"].unique()
    df_movies["Genre {}".format(g)] = df_movies["id"].isin(mids).astype("int16")
df_movies.head()
# Podria ponderar por la cantidad de generos que tiene, si tiene 4 generos cada uno seria 0.25

,id,title,year,imdbID,rtID,Genre Action,Genre Adventure,Genre Animation,Genre Children,Genre Comedy,...,Genre Horror,Genre Mystery,Genre Sci-Fi,Genre IMAX,Genre Documentary,Genre War,Genre Musical,Genre Film-Noir,Genre Western,Genre Short
0,1,Toy story,1995,114709,toy_story,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995,113497,1068044-jumanji,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpy Old Men,1993,107050,grumpy_old_men,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale,1995,114885,waiting_to_exhale,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II,1995,113041,father_of_the_bride_part_ii,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## Perfil de los usuarios

In [7]:
df_ratings.head()

,ID,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,1,51198,608,5.0,17,9,1997,23,40,20
1,2,51198,1073,4.0,17,9,1997,23,41,6
2,3,51198,141,4.0,17,9,1997,23,41,58
3,4,51198,356,4.0,17,9,1997,23,43,44
4,5,51198,1198,5.0,17,9,1997,23,44,35


In [8]:
df_ratings["userID"].unique()

array([51198, 29202, 32283, ..., 62354, 62394, 62847])

In [9]:
# Voy a crea el perfil de un usuario
USUARIO = 75
df_ratings_usuario = df_ratings[df_ratings["userID"] == USUARIO]
df_ratings_usuario.sort_values("rating").head()

,ID,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
473065,525416,75,6225,0.5,29,10,2006,23,21,25
473058,525408,75,3,1.0,29,10,2006,23,17,16
473060,525410,75,2054,1.5,29,10,2006,23,17,33
473081,525435,75,3258,1.5,29,10,2006,23,26,8
473054,525403,75,160,2.0,29,10,2006,23,16,52


In [67]:
# Ahora lo que busco es condensar sus gustos de pelicula por los generos.
# Mergeo al rating de un usuario el perfil de cada pelicula
df_ratings_usuario = df_ratings[df_ratings["userID"] == USUARIO].merge(df_movies, left_on="movieID", right_on="id")
df_ratings_usuario.sort_values("rating").head()

,ID,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second,...,Genre Horror,Genre Mystery,Genre Sci-Fi,Genre IMAX,Genre Documentary,Genre War,Genre Musical,Genre Film-Noir,Genre Western,Genre Short
13,525416,75,6225,0.5,29,10,2006,23,21,25,...,0,0,0,0,0,0,0,0,0,0
6,525408,75,3,1.0,29,10,2006,23,17,16,...,0,0,0,0,0,0,0,0,0,0
8,525410,75,2054,1.5,29,10,2006,23,17,33,...,0,0,1,0,0,0,0,0,0,0
27,525435,75,3258,1.5,29,10,2006,23,26,8,...,0,0,0,0,0,0,0,0,0,0
2,525403,75,160,2.0,29,10,2006,23,16,52,...,0,1,1,0,0,0,0,0,0,0


### Ahora lo que busco es agrupar toda esta tabla en una sola lista, donde pueda sumar los generos para tener para un solo usuario el perfil completo.

In [18]:
# Veo para el usuario 75 que porcentaje tiene sobre los generos Action, Comedy y Drama de las peliculas que opino.
df_ratings_usuario["Genre Action"].mean(), df_ratings_usuario["Genre Comedy"].mean(), df_ratings_usuario["Genre Drama"].mean()

(0.7333333333333333, 0.3111111111111111, 0.3333333333333333)

In [74]:
# Una mejor manera de ranquear, es tomar el minimo y el maximo raking que el usuario puso y no el 1-5 que es el estandar. Si un usuario siempre vota de 3-5, entonces se usa esa base.

(df_ratings_usuario["Genre Action"] * df_ratings_usuario["rating"]).mean()


2.6666666666666665

In [83]:
# Si saco la media, me da CERO en los que no tiene accion y un numero de votacion sobre los que si tenian accion. Idealmente no deberia contar los CEROs.
# Lo Normalizo por la cantidad de votaciones posibles
df_ratings_usuario.loc[df_ratings_usuario["Genre Action"]==1, "rating"].mean() / 5.0, df_ratings_usuario.loc[df_ratings_usuario["Genre Drama"]==1, "rating"].mean() / 5.0, df_ratings_usuario.loc[df_ratings_usuario["Genre Comedy"]==1, "rating"].mean() / 5.0

(0.7272727272727273, 0.7533333333333333, 0.6357142857142857)

### Esto va generado varios vectores entre CERO y UNO y el vector del usuario. Luego deberia buscar cual es el vector mas cercano.

In [85]:
# Perfil del USER 75, la idea es generar el perfil para los usuarios.
# Creo PERFIL como diccionario
PERFIL ={}
for c in df_ratings_usuario.columns:
    # Me quedo solo con las columnas que empiezan con GENRE 
    if not c.startswith("Genre"):
       continue

    PERFIL[c] = [df_ratings_usuario.loc[df_ratings_usuario[c]==1, "rating"].mean() / 5.0]

print(PERFIL)



{'Genre Action': [0.7272727272727273], 'Genre Adventure': [0.6571428571428571], 'Genre Animation': [0.9], 'Genre Children': [0.3], 'Genre Comedy': [0.6357142857142857], 'Genre Fantasy': [0.625], 'Genre Romance': [0.5], 'Genre Drama': [0.7533333333333333], 'Genre Crime': [0.8300000000000001], 'Genre Thriller': [0.7761904761904762], 'Genre Horror': [0.7], 'Genre Mystery': [0.55], 'Genre Sci-Fi': [0.6538461538461539], 'Genre IMAX': [nan], 'Genre Documentary': [nan], 'Genre War': [0.7333333333333333], 'Genre Musical': [0.9], 'Genre Film-Noir': [nan], 'Genre Western': [0.5], 'Genre Short': [nan]}


In [86]:
# Lo convierto en un DataFrame
# fillna(0) me llena los nan como CERO
df_perfil = pd.DataFrame.from_dict(PERFIL).fillna(0)
df_perfil

,Genre Action,Genre Adventure,Genre Animation,Genre Children,Genre Comedy,Genre Fantasy,Genre Romance,Genre Drama,Genre Crime,Genre Thriller,Genre Horror,Genre Mystery,Genre Sci-Fi,Genre IMAX,Genre Documentary,Genre War,Genre Musical,Genre Film-Noir,Genre Western,Genre Short
0,0.727273,0.657143,0.9,0.3,0.635714,0.625,0.5,0.753333,0.83,0.77619,0.7,0.55,0.653846,0.0,0.0,0.733333,0.9,0.0,0.5,0.0


### Este es el perfil del usuario 75, e informa segun lo que califico cuales son los generos que estan mas cerca de su gusto.
### Ahora lo que busco es cuales son las peliculas que estan mas cerca de este usuario usando la Similitud Coseno (1 es super similar)

In [91]:
# Agarro una pelicula cualquier y me fijo que tan cerca o lejos esta de su perfil.
MOVIE = 65088
movie = df_movies.query(f"id=={MOVIE}")
columnas_genero = [c for c in df_movies.columns if c.startswith("Genre")]
A = movie[columnas_genero].values[0]
B = df_perfil[columnas_genero].values[0]
# A es la pelicula y B es el perfil del usuario

In [92]:
# El perfil
# Similaridad Coseno, mientras mas cerca del 1 mejor
np.dot(A,B) / (np.linalg.norm(A) * np.linalg.norm(B))

0.33397865808365

In [ ]:
# Ahora busco hacerlo para todas las peliculas

columnas_genero = [c for c in df_movies.columns if c.startswith("Genre")]
B = df_perfil[columnas_genero].values[0]

for index, row in df_movies.iterrows():    
    movie = df_movies.query("id == {}".format(row["id"]))

    A = movie[columnas_genero].values[0]
    #print(row["id"], np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B)))

#0.49472772819267147 * 5.0
# Si tomo el mayor y lo multiplico por 5, eso me da el rating.

# Tambien podria tomar las 5 con mayor rating y podria decir que esas son las 5 que mas le deberian gustar.
# No deberia mostrarle peliculas que YA vio, porque esas podrian ser muy cercanas.
# Esta tecnica me permite con 1 sola pelicula, ya empezar a recomendarle peliculas similares.

In [ ]:
# Con el pais podria hacer algo similar y decir para cada usuario que pais de pelicula prefiere. Pero quizas no haya que hacer * 5.0?
# Podria pondear los generos por 0.6 y los paises por 0.4 ...
# Como reflejar en el perfil del usuario que prefiere peliculas a partir del año 2000?
# Se podrian hacer bines por decadas. Agregar el año en el perfil, etc. 
# (año- minimoAño / maximoAño - minimoAños) los paso a CERO y UNO, y mutiplico año contra año.
# Podria incluir
# Año, directores, actores, paises, etc.

## Calculo para todos los usuarios

In [7]:
df_movies.head()

,id,title,year,imdbID,rtID,Genre Action,Genre Adventure,Genre Animation,Genre Children,Genre Comedy,...,Genre Horror,Genre Mystery,Genre Sci-Fi,Genre IMAX,Genre Documentary,Genre War,Genre Musical,Genre Film-Noir,Genre Western,Genre Short
0,1,Toy story,1995,114709,toy_story,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995,113497,1068044-jumanji,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpy Old Men,1993,107050,grumpy_old_men,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale,1995,114885,waiting_to_exhale,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II,1995,113041,father_of_the_bride_part_ii,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df_ratings["userID"].unique()

array([51198, 29202, 32283, ..., 62354, 62394, 62847])

In [170]:
def perfil_por_usuario(USUARIO):

    # Voy a crea el perfil de un usuario
    df_ratings_generic = df_ratings[df_ratings["userID"] == USUARIO]
    df_ratings_generic.sort_values("rating").head()
    # Ahora lo que busco es condensar sus gustos de pelicula por los generos.
    # Mergeo al rating de un usuario el perfil de cada pelicula
    df_ratings_generic = df_ratings[df_ratings["userID"] == USUARIO].merge(df_movies, left_on="movieID", right_on="id")
    df_ratings_generic.sort_values("rating").head()

    # Perfil del USER 29202, la idea es generar el perfil para los usuarios.
    # Creo PERFIL como diccionario
    PERFIL ={}
    for c in df_ratings_generic.columns:
        # Me quedo solo con las columnas que empiezan con GENRE 
        if  c.startswith("userID"):
           PERFIL[c] = [USUARIO] 
        #if not c.startswith(("Genre", "userID")):
        if not c.startswith("Genre"):
            continue
        PERFIL[c] = [df_ratings_generic.loc[df_ratings_generic[c]==1, "rating"].mean() / 5.0]


    #print(PERFIL)
    return PERFIL

In [171]:
df_perfil_full=None
USUARIO = 29202
perfil_por_usuario(USUARIO)


# Lo convierto en un DataFrame
# fillna(0) me llena los nan como CERO
#df_perfil = pd.DataFrame.from_dict(PERFIL).fillna(0)
#df_perfil.insert(0,'userID',USUARIO)


{'userID': [29202],
 'Genre Action': [0.5125],
 'Genre Adventure': [0.5031818181818182],
 'Genre Animation': [0.4470588235294118],
 'Genre Children': [0.4333333333333333],
 'Genre Comedy': [0.4436131386861314],
 'Genre Fantasy': [0.492741935483871],
 'Genre Romance': [0.5364077669902912],
 'Genre Drama': [0.5667250437828371],
 'Genre Crime': [0.5349112426035503],
 'Genre Thriller': [0.5311797752808989],
 'Genre Horror': [0.4645161290322581],
 'Genre Mystery': [0.54875],
 'Genre Sci-Fi': [0.5093567251461988],
 'Genre IMAX': [0.6],
 'Genre Documentary': [0.6533333333333333],
 'Genre War': [0.5755555555555556],
 'Genre Musical': [0.5045454545454546],
 'Genre Film-Noir': [0.4428571428571429],
 'Genre Western': [0.5565217391304348],
 'Genre Short': [nan]}

In [429]:
IDs = ['29202','75','62354']
df_addoc = df_ratings.loc[df_ratings["userID"].isin(IDs)]
df_addoc = df_ratings
df_addoc["userID"].unique()

array([51198, 29202, 32283, ..., 62354, 62394, 62847])

In [446]:

#df_perfil = pd.DataFrame.from_dict(PERFIL).fillna(0)
#df_perfil.insert(0,'userID',USUARIO)
df_full_profile = pd.DataFrame()
for g in df_addoc["userID"].unique():
    #PERFIl_FULL = perfil_por_usuario(g)
    df_full_profile = df_full_profile.append(pd.DataFrame.from_dict(perfil_por_usuario(g)).fillna(0), ignore_index=True)
    #print(df_full_profile.tail(1))


In [447]:
df_full_profile

,userID,Genre Action,Genre Adventure,Genre Animation,Genre Children,Genre Comedy,Genre Fantasy,Genre Romance,Genre Drama,Genre Crime,...,Genre Horror,Genre Mystery,Genre Sci-Fi,Genre IMAX,Genre Documentary,Genre War,Genre Musical,Genre Film-Noir,Genre Western,Genre Short
0,51198,0.783824,0.798000,0.788889,0.766667,0.779000,0.796667,0.810811,0.833028,0.865116,...,0.728571,0.804762,0.784211,0.9,0.883333,0.816667,0.840000,0.800000,0.733333,0.0
1,29202,0.512500,0.503182,0.447059,0.433333,0.443613,0.492742,0.536408,0.566725,0.534911,...,0.464516,0.548750,0.509357,0.6,0.653333,0.575556,0.504545,0.442857,0.556522,0.0
2,32283,0.687004,0.694545,0.806452,0.758333,0.694751,0.694805,0.723832,0.739817,0.728571,...,0.662162,0.742857,0.656566,0.8,0.828571,0.721212,0.700000,0.800000,0.769565,0.0
3,36495,0.673034,0.677551,0.763636,0.742857,0.764198,0.631579,0.740541,0.755914,0.767241,...,0.800000,0.725000,0.640000,0.0,0.500000,0.762500,0.800000,1.000000,0.600000,0.0
4,28350,0.653191,0.666667,0.705769,0.658915,0.632463,0.656180,0.670940,0.669822,0.635714,...,0.429412,0.615000,0.597561,0.8,0.000000,0.684211,0.641667,0.700000,0.635294,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2108,61985,0.749457,0.742424,0.746154,0.733333,0.709677,0.738095,0.731579,0.761538,0.735849,...,0.738095,0.816216,0.741509,0.9,0.800000,0.768421,0.685714,0.750000,0.725000,0.0
2109,62042,0.583333,0.693333,0.690000,0.693333,0.601818,0.685714,0.659375,0.763265,0.671429,...,0.566667,0.700000,0.516667,0.0,0.000000,0.850000,0.638462,0.000000,0.800000,0.0
2110,62354,0.827273,0.821429,0.820000,0.840000,0.726316,0.888889,0.768750,0.809677,0.900000,...,0.660000,0.700000,0.840000,0.0,0.000000,0.750000,0.733333,1.000000,0.700000,0.0
2111,62394,0.607477,0.619178,0.628571,0.561111,0.557534,0.613889,0.611667,0.684817,0.708696,...,0.631373,0.664706,0.516364,0.9,0.500000,0.737037,0.650000,0.733333,0.607692,0.0


## En df_full_profile tengo el perfil de todos los usuarios y cada linea representa segun lo que califico un usuario que generos estan mas cerca o mas lejos de sus preferencia.
## Ahora lo que busco es cuales son las peliculas que estan mas cerca de casa usuario usando la Similitud Coseno (1 es super similar)

In [467]:
df_movies_adDoc = df_movies[0:20]

In [468]:
# Dado un perfil de usuario, veo que pelicula es la mas afin a sus votaciones.

def pelicula_afin_a_perfil(userID, B, list_profile):
    columnas_genero = [c for c in df_movies_adDoc.columns if c.startswith("Genre")]
    
    similarityC_base=0
    movieID = 0
    for index, row in (df_movies_adDoc.iterrows()):    
        movie = df_movies_adDoc.query("id == {}".format(row["id"]))
        A = movie[columnas_genero].values[0]
        similarityC = np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))
        if (similarityC_base < simCosin):
            similarityC_base = simCosin
            movieID = row["id"]
        #RANKING[index] = [row["id"], simCosin]
    
    user_row = dict()
    user_row["userID"] = userID
    user_row["movieID"] = row["id"]
    user_row["similarityC"] = similarityC_base
    list_profile.append(user_row)
    return list_profile

In [421]:
#print("movieID:", movieID, "simCosin: ",simCosin_base)

In [419]:
#df_perfil[columnas_genero].values[0]

In [420]:
#COUNT=1
## iloc[row slicing, column slicing]
#B=df_full_profile.iloc[COUNT-1:COUNT, 1:].values[0]
#B

In [427]:
#USER = "51198"
#column_names = ["userID","movieID", "simCos"]
#df_profile_movieSelection = pd.DataFrame(columns = column_names)
#df_profile_movieSelection = []
#pelicula_afin_a_perfil(USER,B,df_profile_movieSelection)

9it [00:00, 157.44it/s]0.49912149585718957



[{'userID': '51198', 'movieID': 9, 'simCos': 0.49912149585718957}]

In [464]:
df_profile_movieSelection = []
COUNT=1
for g in tqdm_notebook(df_full_profile["userID"].unique()):
    B=df_full_profile.iloc[COUNT-1:COUNT, 1:].values[0]
    pelicula_afin_a_perfil(g,B,df_profile_movieSelection)
    COUNT=COUNT+1


In [465]:
#df_profile_movieSelection
dataset = pd.DataFrame(df_profile_movieSelection)

7983474454},
 {'userID': 68393, 'movieID': 9, 'simCos': 0.5413912514048348},
 {'userID': 68424, 'movieID': 9, 'simCos': 0.5442218924462051},
 {'userID': 68829, 'movieID': 9, 'simCos': 0.505504465028736},
 {'userID': 68947, 'movieID': 9, 'simCos': 0.4588278829319902},
 {'userID': 69153, 'movieID': 9, 'simCos': 0.49332820359308666},
 {'userID': 69716, 'movieID': 9, 'simCos': 0.5150577737637642},
 {'userID': 70313, 'movieID': 9, 'simCos': 0.5334050879803173},
 {'userID': 70369, 'movieID': 9, 'simCos': 0.5103523369443143},
 {'userID': 70525, 'movieID': 9, 'simCos': 0.5311788270011929},
 {'userID': 70542, 'movieID': 9, 'simCos': 0.5437679957029553},
 {'userID': 70981, 'movieID': 9, 'simCos': 0.5292066392962351},
 {'userID': 71478, 'movieID': 9, 'simCos': 0.5246029105055355},
 {'userID': 175, 'movieID': 9, 'simCos': 0.5292409307100597},
 {'userID': 325, 'movieID': 9, 'simCos': 0.5113305982164427},
 {'userID': 637, 'movieID': 9, 'simCos': 0.5363865253112835},
 {'userID': 1547, 'movieID': 9, '